## **Import Libraries** 

In [93]:
import pandas as pd 
import numpy as np
import plotly.express as px

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import KFold
from sklearn.impute import SimpleImputer

import os  
import gc

from google.colab import drive

## **Functions**

### **Helper Functions** 

In [21]:
def reduce_memory_usage(df):
    """
        This function reduces the memory usage of the dataframes 
    """
  
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [22]:
def missing_values_dataframe(df):
    """
    Create missing values dataframe 
    df : the input dataframe 
    categorical_columns : the default value is none if you want to change all the object cols to one hot encoding
    nan_as_category : treat the nan values as a class in the categorical columns

    returns:
    mis_val_table_ren_columns : a dataframe with number of missing values of each feature and the percentage of this value 
    """
    # Total missing values
    mis_val = df.isnull().sum()
    
    # Percentage of missing values
    mis_val_percent = 100 * df.isnull().sum() / len(df)
    
    # Make a table with the results
    mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
    
    # Rename the columns
    mis_val_table_ren_columns = mis_val_table.rename(
    columns = {0 : 'Number of Missing Values', 1 : '% of Missing Values'})
    
    # Sort the table by percentage of missing descending
    mis_val_table_ren_columns = mis_val_table_ren_columns[
        mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
    '% of Missing Values', ascending=False).round(1)
    
    # Print some summary information
    print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
        "There are " + str(mis_val_table_ren_columns.shape[0]) +
          " columns that have missing values.")
    
    # Return the dataframe with missing information
    return mis_val_table_ren_columns

In [23]:
def one_hot_encoding(df, categorical_columns=None, nan_as_category=True):
    """
    Create a new column for each categorical value in categorical columns using get_dummies function. 
    df : the input dataframe 
    categorical_columns : the default value is none if you want to change all the object cols to one hot encoding
    nan_as_category : treat the nan values as a class in the categorical columns
    
    returns:
    df : the converted version of the input dataframe 
    categorical_columns : a list of categorical columns names 
    """
    original_columns = list(df.columns)
    # get the categorical columns names in the dataframe 
    if not categorical_columns:
        categorical_columns = [col for col in df.columns if df[col].dtype == 'object']
    df = pd.get_dummies(df, columns=categorical_columns, dummy_na=nan_as_category)
    categorical_columns = [c for c in df.columns if c not in original_columns]
    return df, categorical_columns

In [24]:
def label_encoding(df, categorical_columns=None):
    """
    Encode categorical values as integers (0,1,2,3...)
    df : the input dataframe  
    categorical_columns : the default value is none if you want to change all the object cols to label encode

    returns: 
    df : the converted version of the input dataframe 
    categorical_columns :  list of categorical columns names 
    """
    if not categorical_columns:
        categorical_columns = [col for col in df.columns if df[col].dtype == 'object']
    for col in categorical_columns:
        df[col], uniques = pd.factorize(df[col])
    return df, categorical_columns

In [25]:
def do_mean(df, group_cols, counted, agg_name):
    """
    aggregate over mean of selected columns 
    """
    gp = df[group_cols + [counted]].groupby(group_cols)[counted].mean().reset_index().rename(
        columns={counted: agg_name})
    df = df.merge(gp, on=group_cols, how='left')
    del gp
    gc.collect()
    return df

In [26]:
def do_median(df, group_cols, counted, agg_name):
    """
    aggregate over median of selected columns 
    """
    gp = df[group_cols + [counted]].groupby(group_cols)[counted].median().reset_index().rename(
        columns={counted: agg_name})
    df = df.merge(gp, on=group_cols, how='left')
    del gp
    gc.collect()
    return df

In [27]:
def do_std(df, group_cols, counted, agg_name):
    """
    aggregate over std of selected columns 
    """
    gp = df[group_cols + [counted]].groupby(group_cols)[counted].std().reset_index().rename(
        columns={counted: agg_name})
    df = df.merge(gp, on=group_cols, how='left')
    del gp
    gc.collect()
    return df

In [28]:
def do_sum(df, group_cols, counted, agg_name):
    """
    aggregate over sum of selected columns 
    """
    gp = df[group_cols + [counted]].groupby(group_cols)[counted].sum().reset_index().rename(
        columns={counted: agg_name})
    df = df.merge(gp, on=group_cols, how='left')
    del gp
    gc.collect()
    return df

In [29]:
def group(df_to_agg, prefix, aggregations, aggregate_by= 'SK_ID_CURR'):
    agg_df = df_to_agg.groupby(aggregate_by).agg(aggregations)
    agg_df.columns = pd.Index(['{}{}_{}'.format(prefix, e[0], e[1].upper())
                               for e in agg_df.columns.tolist()])
    return agg_df.reset_index()

In [30]:
def group_and_merge(df_to_agg, df_to_merge, prefix, aggregations, aggregate_by= 'SK_ID_CURR'):
    agg_df = group(df_to_agg, prefix, aggregations, aggregate_by= aggregate_by)
    return df_to_merge.merge(agg_df, how='left', on= aggregate_by)

### **application train preprocessing** 

In [57]:
def application_train_preprocessing(df):
    """ Process application_train.csv and application_test.csv and return a pandas dataframe. """

    # Data cleaning
    #remove some data that isn't in the test dataset 
    df = df[df['CODE_GENDER'] != 'XNA']  # 4 people with XNA code gender
    df = df[df['NAME_INCOME_TYPE'] != 'Maternity leave']
    df = df[df['NAME_FAMILY_STATUS'] != 'Unknown›.']

    #handling outliers 
    df = df[df['AMT_INCOME_TOTAL'] < 20000000]  # Max income in test is 4M; train has a 117M value
    
    #Null valuse 
    df['DAYS_EMPLOYED'].replace(365243, np.nan, inplace=True)
    df['DAYS_LAST_PHONE_CHANGE'].replace(0, np.nan, inplace=True)
    
    # Categorical age - based on target=1 plot
    df['AGE_RANGE'] = df['DAYS_BIRTH'].apply(lambda x: get_age_label(x))

    # New features based on External sources
    df['EXT_SOURCES_PROD'] = df['EXT_SOURCE_1'] * df['EXT_SOURCE_2'] * df['EXT_SOURCE_3']
    df['EXT_SOURCES_WEIGHTED'] = df.EXT_SOURCE_1 * 2 + df.EXT_SOURCE_2 * 1 + df.EXT_SOURCE_3 * 3
    for function_name in ['min', 'max', 'mean', 'nanmedian', 'var']:
        feature_name = 'EXT_SOURCES_{}'.format(function_name.upper())
        df[feature_name] = eval('np.{}'.format(function_name))(
            df[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']], axis=1)

    # Credit ratios
    df['CREDIT_TO_ANNUITY_RATIO'] = df['AMT_CREDIT'] / df['AMT_ANNUITY']
    df['CREDIT_TO_GOODS_RATIO'] = df['AMT_CREDIT'] / df['AMT_GOODS_PRICE']

    # Income ratios
    df['ANNUITY_TO_INCOME_RATIO'] = df['AMT_ANNUITY'] / df['AMT_INCOME_TOTAL']
    df['CREDIT_TO_INCOME_RATIO'] = df['AMT_CREDIT'] / df['AMT_INCOME_TOTAL']
    df['INCOME_TO_EMPLOYED_RATIO'] = df['AMT_INCOME_TOTAL'] / df['DAYS_EMPLOYED']
    df['INCOME_TO_BIRTH_RATIO'] = df['AMT_INCOME_TOTAL'] / df['DAYS_BIRTH']

    # Time ratios
    df['EMPLOYED_TO_BIRTH_RATIO'] = df['DAYS_EMPLOYED'] / df['DAYS_BIRTH']
    df['ID_TO_BIRTH_RATIO'] = df['DAYS_ID_PUBLISH'] / df['DAYS_BIRTH']
    df['CAR_TO_BIRTH_RATIO'] = df['OWN_CAR_AGE'] / df['DAYS_BIRTH']
    df['CAR_TO_EMPLOYED_RATIO'] = df['OWN_CAR_AGE'] / df['DAYS_EMPLOYED']
    df['PHONE_TO_BIRTH_RATIO'] = df['DAYS_LAST_PHONE_CHANGE'] / df['DAYS_BIRTH']

    # Groupby: Statistics for applications in the same group
    group = ['ORGANIZATION_TYPE', 'NAME_EDUCATION_TYPE', 'OCCUPATION_TYPE', 'AGE_RANGE', 'CODE_GENDER']
    df = do_median(df, group, 'EXT_SOURCES_MEAN', 'GROUP_EXT_SOURCES_MEDIAN')
    df = do_std(df, group, 'EXT_SOURCES_MEAN', 'GROUP_EXT_SOURCES_STD')
    df = do_mean(df, group, 'AMT_INCOME_TOTAL', 'GROUP_INCOME_MEAN')
    df = do_std(df, group, 'AMT_INCOME_TOTAL', 'GROUP_INCOME_STD')
    df = do_mean(df, group, 'CREDIT_TO_ANNUITY_RATIO', 'GROUP_CREDIT_TO_ANNUITY_MEAN')
    df = do_std(df, group, 'CREDIT_TO_ANNUITY_RATIO', 'GROUP_CREDIT_TO_ANNUITY_STD')
    df = do_mean(df, group, 'AMT_CREDIT', 'GROUP_CREDIT_MEAN')
    df = do_mean(df, group, 'AMT_ANNUITY', 'GROUP_ANNUITY_MEAN')
    df = do_std(df, group, 'AMT_ANNUITY', 'GROUP_ANNUITY_STD')

    # Encode categorical features (LabelEncoder)
    df, le_encoded_cols = label_encoding(df, None)

    df = drop_application_columns(df)

    return df

In [58]:
def drop_application_columns(df):
    """ Drop features based on permutation feature importance. """
    drop_list = [
        'CNT_CHILDREN', 'CNT_FAM_MEMBERS', 'HOUR_APPR_PROCESS_START',
        'FLAG_EMP_PHONE', 'FLAG_MOBIL', 'FLAG_CONT_MOBILE', 'FLAG_EMAIL', 'FLAG_PHONE',
        'FLAG_OWN_REALTY', 'REG_REGION_NOT_LIVE_REGION', 'REG_REGION_NOT_WORK_REGION',
        'REG_CITY_NOT_WORK_CITY', 'OBS_30_CNT_SOCIAL_CIRCLE', 'OBS_60_CNT_SOCIAL_CIRCLE',
        'AMT_REQ_CREDIT_BUREAU_DAY', 'AMT_REQ_CREDIT_BUREAU_MON', 'AMT_REQ_CREDIT_BUREAU_YEAR', 
        'COMMONAREA_MODE', 'NONLIVINGAREA_MODE', 'ELEVATORS_MODE', 'NONLIVINGAREA_AVG',
        'FLOORSMIN_MEDI', 'LANDAREA_MODE', 'NONLIVINGAREA_MEDI', 'LIVINGAPARTMENTS_MODE',
        'FLOORSMIN_AVG', 'LANDAREA_AVG', 'FLOORSMIN_MODE', 'LANDAREA_MEDI',
        'COMMONAREA_MEDI', 'YEARS_BUILD_AVG', 'COMMONAREA_AVG', 'BASEMENTAREA_AVG',
        'BASEMENTAREA_MODE', 'NONLIVINGAPARTMENTS_MEDI', 'BASEMENTAREA_MEDI', 
        'LIVINGAPARTMENTS_AVG', 'ELEVATORS_AVG', 'YEARS_BUILD_MEDI', 'ENTRANCES_MODE',
        'NONLIVINGAPARTMENTS_MODE', 'LIVINGAREA_MODE', 'LIVINGAPARTMENTS_MEDI',
        'YEARS_BUILD_MODE', 'YEARS_BEGINEXPLUATATION_AVG', 'ELEVATORS_MEDI', 'LIVINGAREA_MEDI',
        'YEARS_BEGINEXPLUATATION_MODE', 'NONLIVINGAPARTMENTS_AVG', 'HOUSETYPE_MODE',
        'FONDKAPREMONT_MODE', 'EMERGENCYSTATE_MODE'
    ]
    
    df.drop(drop_list, axis=1, inplace=True)
    
    return df

In [59]:
def get_age_label(days_birth):
    """ Return the age group label (int). """
    age_years = -days_birth / 365
    if age_years < 27: 
      return 1
    elif age_years < 40: 
      return 2
    elif age_years < 50: 
      return 3
    elif age_years < 65: 
      return 4
    elif age_years < 99: 
      return 5
    else: return 0

### **bureau and bureau balance preprocessing** 

In [71]:
def bureau_and_balance(bureau, bureau_balance, nan_as_category = True):
    bb, bb_cat = one_hot_encoding(bureau_balance, nan_as_category= False)
    bureau, bureau_cat = one_hot_encoding(bureau, nan_as_category= False)
    
    # Bureau balance: Perform aggregations and merge with bureau.csv
    bb_aggregations = {'MONTHS_BALANCE': ['min', 'max', 'size']}
    for col in bb_cat:
        bb_aggregations[col] = ['mean']
    bb_agg = bb.groupby('SK_ID_BUREAU').agg(bb_aggregations)
    bb_agg.columns = pd.Index([e[0] + "_" + e[1].upper() for e in bb_agg.columns.tolist()])
    bureau = bureau.join(bb_agg, how='left', on='SK_ID_BUREAU')
    bureau.drop(['SK_ID_BUREAU'], axis=1, inplace= True)
    del bb, bb_agg
    gc.collect()
    
    # Bureau and bureau_balance numeric features
    num_aggregations = {
        'DAYS_CREDIT': ['min', 'max', 'mean', 'var'],
        'DAYS_CREDIT_ENDDATE': ['min', 'max', 'mean'],
        'DAYS_CREDIT_UPDATE': ['mean'],
        'CREDIT_DAY_OVERDUE': ['max', 'mean'],
        'AMT_CREDIT_MAX_OVERDUE': ['mean'],
        'AMT_CREDIT_SUM': ['max', 'mean', 'sum'],
        'AMT_CREDIT_SUM_DEBT': ['max', 'mean', 'sum'],
        'AMT_CREDIT_SUM_OVERDUE': ['mean'],
        'AMT_CREDIT_SUM_LIMIT': ['mean', 'sum'],
        'AMT_ANNUITY': ['max', 'mean'],
        'CNT_CREDIT_PROLONG': ['sum'],
        'MONTHS_BALANCE_MIN': ['min'],
        'MONTHS_BALANCE_MAX': ['max'],
        'MONTHS_BALANCE_SIZE': ['mean', 'sum']
    }
    # Bureau and bureau_balance categorical features
    cat_aggregations = {}
    for cat in bureau_cat: cat_aggregations[cat] = ['mean']
    for cat in bb_cat: cat_aggregations[cat + "_MEAN"] = ['mean']
    
    bureau_agg = bureau.groupby('SK_ID_CURR').agg({**num_aggregations, **cat_aggregations})
    bureau_agg.columns = pd.Index(['BURO_' + e[0] + "_" + e[1].upper() for e in bureau_agg.columns.tolist()])
    # Bureau: Active credits - using only numerical aggregations
    active = bureau[bureau['CREDIT_ACTIVE_Active'] == 1]
    active_agg = active.groupby('SK_ID_CURR').agg(num_aggregations)
    active_agg.columns = pd.Index(['ACTIVE_' + e[0] + "_" + e[1].upper() for e in active_agg.columns.tolist()])
    bureau_agg = bureau_agg.join(active_agg, how='left', on='SK_ID_CURR')
    del active, active_agg
    gc.collect()
    # Bureau: Closed credits - using only numerical aggregations
    closed = bureau[bureau['CREDIT_ACTIVE_Closed'] == 1]
    closed_agg = closed.groupby('SK_ID_CURR').agg(num_aggregations)
    closed_agg.columns = pd.Index(['CLOSED_' + e[0] + "_" + e[1].upper() for e in closed_agg.columns.tolist()])
    bureau_agg = bureau_agg.join(closed_agg, how='left', on='SK_ID_CURR')
    del closed, closed_agg, bureau
    gc.collect()
    return bureau_agg

## **Downloading Data From Kaggle**

In [36]:
!pip install -q kaggle

In [37]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"ahmedgamal96","key":"9bc61648413bac1ff44f847174bb0069"}'}

In [38]:
!mkdir ~/.kaggle

In [39]:
!cp kaggle.json ~/.kaggle/

In [40]:
!chmod 600 ~/.kaggle/kaggle.json

In [41]:
!kaggle datasets list

ref                                                              title                                                size  lastUpdated          downloadCount  voteCount  usabilityRating  
---------------------------------------------------------------  --------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
ahsan81/hotel-reservations-classification-dataset                Hotel Reservations Dataset                          480KB  2023-01-04 12:50:31           7112        249  1.0              
themrityunjaypathak/most-subscribed-1000-youtube-channels        Most Subscribed 1000 Youtube Channels                28KB  2023-01-21 14:42:05           1215         42  1.0              
senapatirajesh/netflix-tv-shows-and-movies                       Latest Netflix TV shows and movies                    1MB  2023-01-14 17:03:12           2604         69  0.9411765        
ulrikthygepedersen/online-retail-dataset               

In [42]:
!kaggle competitions download -c Home-Credit-Default-Risk

100% 685M/688M [00:21<00:00, 41.4MB/s]
100% 688M/688M [00:21<00:00, 33.2MB/s]


In [43]:
!unzip /content/Home-Credit-Default-Risk.zip

Archive:  /content/Home-Credit-Default-Risk.zip
  inflating: HomeCredit_columns_description.csv  
  inflating: POS_CASH_balance.csv    
  inflating: application_test.csv    
  inflating: application_train.csv   
  inflating: bureau.csv              
  inflating: bureau_balance.csv      
  inflating: credit_card_balance.csv  
  inflating: installments_payments.csv  
  inflating: previous_application.csv  
  inflating: sample_submission.csv   


## **Reading Data** 

### **application_train.csv File** 

In [44]:
application_train = reduce_memory_usage(pd.read_csv('/content/application_train.csv'))
application_train

Memory usage of dataframe is 286.23 MB
Memory usage after optimization is: 92.38 MB
Decreased by 67.7%


,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,456251,0,Cash loans,M,N,N,0,157500.0,254700.0,27558.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
307507,456252,0,Cash loans,F,N,Y,0,72000.0,269550.0,12001.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
307508,456253,0,Cash loans,F,N,Y,0,153000.0,677664.0,29979.0,...,0,0,0,0,1.0,0.0,0.0,1.0,0.0,1.0
307509,456254,1,Cash loans,F,N,Y,0,171000.0,370107.0,20205.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


### **bureau.csv File** 

In [45]:
bureau = reduce_memory_usage(pd.read_csv('/content/bureau.csv'))
bureau

Memory usage of dataframe is 222.62 MB
Memory usage after optimization is: 112.95 MB
Decreased by 49.3%


,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,9.132300e+04,0.0,NaN,0.0,Consumer credit,-131,NaN
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,2.250000e+05,171342.0,NaN,0.0,Credit card,-20,NaN
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,4.643235e+05,NaN,NaN,0.0,Consumer credit,-16,NaN
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,9.000000e+04,NaN,NaN,0.0,Credit card,-16,NaN
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2.700000e+06,NaN,NaN,0.0,Consumer credit,-21,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1716423,259355,5057750,Active,currency 1,-44,0,-30.0,NaN,0.0,0,1.125000e+04,11250.0,0.0,0.0,Microloan,-19,NaN
1716424,100044,5057754,Closed,currency 1,-2648,0,-2432.0,-2492.0,5476.5,0,3.813084e+04,0.0,0.0,0.0,Consumer credit,-2493,NaN
1716425,100044,5057762,Closed,currency 1,-1809,0,-1628.0,-970.0,NaN,0,1.557000e+04,NaN,NaN,0.0,Consumer credit,-967,NaN
1716426,246829,5057770,Closed,currency 1,-1878,0,-1513.0,-1513.0,NaN,0,3.600000e+04,0.0,0.0,0.0,Consumer credit,-1508,NaN


### **bureau_balance.csv File** 

In [46]:
bureau_balance = reduce_memory_usage(pd.read_csv('/content/bureau_balance.csv'))
bureau_balance

Memory usage of dataframe is 624.85 MB
Memory usage after optimization is: 338.46 MB
Decreased by 45.8%


,SK_ID_BUREAU,MONTHS_BALANCE,STATUS
0,5715448,0,C
1,5715448,-1,C
2,5715448,-2,C
3,5715448,-3,C
4,5715448,-4,C
...,...,...,...
27299920,5041336,-47,X
27299921,5041336,-48,X
27299922,5041336,-49,X
27299923,5041336,-50,X


## **Data Preprocessing and Feature Engineering** 

### **Null Values** 

In [50]:
missing_values_df = missing_values_dataframe(application_train)

Your selected dataframe has 122 columns.
There are 67 columns that have missing values.


In [51]:
missing_values_df.head(10)

,Number of Missing Values,% of Missing Values
COMMONAREA_MEDI,214865,69.9
COMMONAREA_AVG,214865,69.9
COMMONAREA_MODE,214865,69.9
NONLIVINGAPARTMENTS_MEDI,213514,69.4
NONLIVINGAPARTMENTS_MODE,213514,69.4
NONLIVINGAPARTMENTS_AVG,213514,69.4
FONDKAPREMONT_MODE,210295,68.4
LIVINGAPARTMENTS_MODE,210199,68.4
LIVINGAPARTMENTS_MEDI,210199,68.4
LIVINGAPARTMENTS_AVG,210199,68.4


In [52]:
missing_values_df_3 = missing_values_dataframe(bureau)

Your selected dataframe has 17 columns.
There are 7 columns that have missing values.


In [53]:
missing_values_df_3.head(10)

,Number of Missing Values,% of Missing Values
AMT_ANNUITY,1226791,71.5
AMT_CREDIT_MAX_OVERDUE,1124488,65.5
DAYS_ENDDATE_FACT,633653,36.9
AMT_CREDIT_SUM_LIMIT,591780,34.5
AMT_CREDIT_SUM_DEBT,257669,15.0
DAYS_CREDIT_ENDDATE,105553,6.1
AMT_CREDIT_SUM,13,0.0


In [54]:
missing_values_df_4 = missing_values_dataframe(bureau_balance)

Your selected dataframe has 3 columns.
There are 0 columns that have missing values.


In [55]:
missing_values_df_4.head(10)

,Number of Missing Values,% of Missing Values


### **Applying Data Preprocessing Pipelines** 

In [60]:
df = application_train_preprocessing(application_train)

/usr/local/lib/python3.8/dist-packages/numpy/lib/nanfunctions.py:1119: RuntimeWarning: All-NaN slice encountered
  r, k = function_base._ureduce(a, func=_nanmedian, axis=axis, out=out,


In [61]:
df.head(10)

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,...,PHONE_TO_BIRTH_RATIO,GROUP_EXT_SOURCES_MEDIAN,GROUP_EXT_SOURCES_STD,GROUP_INCOME_MEAN,GROUP_INCOME_STD,GROUP_CREDIT_TO_ANNUITY_MEAN,GROUP_CREDIT_TO_ANNUITY_STD,GROUP_CREDIT_MEAN,GROUP_ANNUITY_MEAN,GROUP_ANNUITY_STD
0,100002,1,0,0,0,202500.0,406597.5,24700.5,351000.0,0,...,0.119860,0.362549,0.144436,157996.156250,58123.152983,17.797691,6.330185,426707.9375,23984.642578,11857.838335
1,100003,0,0,1,0,270000.0,1293502.5,35698.5,1129500.0,1,...,0.049389,0.588379,0.123462,170040.125000,69070.194992,23.919558,8.301189,719365.4375,29175.029297,14732.028298
2,100004,0,1,0,1,67500.0,135000.0,6750.0,135000.0,0,...,0.042791,0.566406,0.150097,166614.890625,77154.808936,23.557327,7.396588,644559.8750,27226.810547,13631.849082
3,100006,0,0,1,0,135000.0,312682.5,29686.5,297000.0,0,...,0.032465,0.568359,0.142238,154604.843750,65163.418066,23.044632,7.710703,620479.1875,26512.000000,12892.178496
4,100007,0,0,0,0,121500.0,513000.0,21865.5,513000.0,0,...,0.055489,0.569336,0.191344,159750.000000,58845.135738,29.238951,8.789658,904500.0000,28931.625000,10551.123924
5,100008,0,0,0,0,99000.0,490495.5,27517.5,454500.0,2,...,0.149696,0.530762,0.141688,177413.171875,76616.338262,21.475183,7.641752,624919.6875,28124.412109,13036.246965
6,100009,0,0,1,1,171000.0,1560726.0,41301.0,1395000.0,0,...,0.113369,0.556641,0.126596,215567.593750,108964.388316,22.370134,8.083694,753811.1875,32972.773438,16490.296477
7,100010,0,0,0,1,360000.0,1530000.0,42075.0,1530000.0,0,...,0.056764,0.569824,0.121790,264593.656250,128751.654061,24.581627,8.519011,815941.5625,32994.000000,13918.552709
8,100011,0,0,1,0,112500.0,1019610.0,33826.5,913500.0,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,100012,0,1,0,0,135000.0,405000.0,20250.0,405000.0,0,...,0.115627,0.483398,0.137748,164571.421875,70507.236784,21.094637,7.639009,561315.4375,26175.642578,12771.076214


In [62]:
df.shape

(307501, 98)

In [65]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 307501 entries, 0 to 307500
Data columns (total 98 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   SK_ID_CURR                    307501 non-null  int32  
 1   TARGET                        307501 non-null  int8   
 2   NAME_CONTRACT_TYPE            307501 non-null  int64  
 3   CODE_GENDER                   307501 non-null  int64  
 4   FLAG_OWN_CAR                  307501 non-null  int64  
 5   AMT_INCOME_TOTAL              307501 non-null  float32
 6   AMT_CREDIT                    307501 non-null  float32
 7   AMT_ANNUITY                   307489 non-null  float32
 8   AMT_GOODS_PRICE               307223 non-null  float32
 9   NAME_TYPE_SUITE               307501 non-null  int64  
 10  NAME_INCOME_TYPE              307501 non-null  int64  
 11  NAME_EDUCATION_TYPE           307501 non-null  int64  
 12  NAME_FAMILY_STATUS            307501 non-nul

In [72]:
bureau = bureau_and_balance(bureau, bureau_balance)

In [73]:
bureau.head(10)

,BURO_DAYS_CREDIT_MIN,BURO_DAYS_CREDIT_MAX,BURO_DAYS_CREDIT_MEAN,BURO_DAYS_CREDIT_VAR,BURO_DAYS_CREDIT_ENDDATE_MIN,BURO_DAYS_CREDIT_ENDDATE_MAX,BURO_DAYS_CREDIT_ENDDATE_MEAN,BURO_DAYS_CREDIT_UPDATE_MEAN,BURO_CREDIT_DAY_OVERDUE_MAX,BURO_CREDIT_DAY_OVERDUE_MEAN,...,CLOSED_AMT_CREDIT_SUM_OVERDUE_MEAN,CLOSED_AMT_CREDIT_SUM_LIMIT_MEAN,CLOSED_AMT_CREDIT_SUM_LIMIT_SUM,CLOSED_AMT_ANNUITY_MAX,CLOSED_AMT_ANNUITY_MEAN,CLOSED_CNT_CREDIT_PROLONG_SUM,CLOSED_MONTHS_BALANCE_MIN_MIN,CLOSED_MONTHS_BALANCE_MAX_MAX,CLOSED_MONTHS_BALANCE_SIZE_MEAN,CLOSED_MONTHS_BALANCE_SIZE_SUM
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,-1572,-49,-735.000000,2.400437e+05,-1329.0,1778.0,82.4375,-93.142857,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,-51.0,0.0,35.0,140.0
100002,-1437,-103,-874.000000,1.861500e+05,-1072.0,780.0,-349.0000,-499.875000,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,-47.0,0.0,15.0,90.0
100003,-2586,-606,-1400.750000,8.277836e+05,-2434.0,1216.0,-544.5000,-816.000000,0,0.0,...,0.0,0.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,0.0
100004,-1326,-408,-867.000000,4.213620e+05,-595.0,-382.0,-488.5000,-532.000000,0,0.0,...,0.0,0.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,0.0
100005,-373,-62,-190.666667,2.634033e+04,-128.0,1324.0,439.2500,-54.333333,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,-12.0,0.0,13.0,13.0
100007,-1149,-1149,-1149.000000,NaN,-783.0,-783.0,-783.0000,-783.000000,0,0.0,...,0.0,0.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,0.0
100008,-1097,-78,-757.333333,3.461203e+05,-853.0,471.0,-391.2500,-611.000000,0,0.0,...,0.0,0.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,0.0
100009,-2882,-239,-1271.500000,4.426300e+05,-2152.0,1402.0,-795.0000,-851.611111,0,0.0,...,0.0,0.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,0.0
100010,-2741,-1138,-1939.500000,1.284804e+06,-928.0,689.0,-119.5000,-578.000000,0,0.0,...,0.0,NaN,0.0,NaN,NaN,0.0,-90.0,-55.0,36.0,36.0


In [74]:
bureau.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 305811 entries, 100001 to 456255
Columns: 112 entries, BURO_DAYS_CREDIT_MIN to CLOSED_MONTHS_BALANCE_SIZE_SUM
dtypes: float16(9), float32(36), float64(63), int16(3), int8(1)
memory usage: 198.6 MB


In [75]:
df = df.join(bureau, how='left', on='SK_ID_CURR')
del bureau
gc.collect()

0

In [76]:
df.head(10)

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,...,CLOSED_AMT_CREDIT_SUM_OVERDUE_MEAN,CLOSED_AMT_CREDIT_SUM_LIMIT_MEAN,CLOSED_AMT_CREDIT_SUM_LIMIT_SUM,CLOSED_AMT_ANNUITY_MAX,CLOSED_AMT_ANNUITY_MEAN,CLOSED_CNT_CREDIT_PROLONG_SUM,CLOSED_MONTHS_BALANCE_MIN_MIN,CLOSED_MONTHS_BALANCE_MAX_MAX,CLOSED_MONTHS_BALANCE_SIZE_MEAN,CLOSED_MONTHS_BALANCE_SIZE_SUM
0,100002,1,0,0,0,202500.0,406597.5,24700.5,351000.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,-47.0,0.0,15.0,90.0
1,100003,0,0,1,0,270000.0,1293502.5,35698.5,1129500.0,1,...,0.0,0.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,0.0
2,100004,0,1,0,1,67500.0,135000.0,6750.0,135000.0,0,...,0.0,0.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,0.0
3,100006,0,0,1,0,135000.0,312682.5,29686.5,297000.0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,0,0,0,121500.0,513000.0,21865.5,513000.0,0,...,0.0,0.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,0.0
5,100008,0,0,0,0,99000.0,490495.5,27517.5,454500.0,2,...,0.0,0.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,0.0
6,100009,0,0,1,1,171000.0,1560726.0,41301.0,1395000.0,0,...,0.0,0.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,0.0
7,100010,0,0,0,1,360000.0,1530000.0,42075.0,1530000.0,0,...,0.0,NaN,0.0,NaN,NaN,0.0,-90.0,-55.0,36.0,36.0
8,100011,0,0,1,0,112500.0,1019610.0,33826.5,913500.0,3,...,0.0,0.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,0.0
9,100012,0,1,0,0,135000.0,405000.0,20250.0,405000.0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### **Clean Data after Aggregation** 

In [77]:
missing_values_df_all = missing_values_dataframe(df)

Your selected dataframe has 210 columns.
There are 160 columns that have missing values.


In [78]:
missing_values_df_all.head(10)

,Number of Missing Values,% of Missing Values
ACTIVE_AMT_ANNUITY_MEAN,246155,80.1
ACTIVE_AMT_ANNUITY_MAX,246155,80.1
CLOSED_AMT_ANNUITY_MEAN,241492,78.5
CLOSED_AMT_ANNUITY_MAX,241492,78.5
ACTIVE_MONTHS_BALANCE_MIN_MIN,230982,75.1
ACTIVE_MONTHS_BALANCE_SIZE_MEAN,230982,75.1
ACTIVE_MONTHS_BALANCE_MAX_MAX,230982,75.1
BURO_AMT_ANNUITY_MAX,227492,74.0
BURO_AMT_ANNUITY_MEAN,227492,74.0
CLOSED_MONTHS_BALANCE_MIN_MIN,226766,73.7


In [81]:
features_to_be_dropped = missing_values_df_all[missing_values_df_all['% of Missing Values'] >= 75]
features_to_be_dropped

,Number of Missing Values,% of Missing Values
ACTIVE_AMT_ANNUITY_MEAN,246155,80.1
ACTIVE_AMT_ANNUITY_MAX,246155,80.1
CLOSED_AMT_ANNUITY_MEAN,241492,78.5
CLOSED_AMT_ANNUITY_MAX,241492,78.5
ACTIVE_MONTHS_BALANCE_MIN_MIN,230982,75.1
ACTIVE_MONTHS_BALANCE_SIZE_MEAN,230982,75.1
ACTIVE_MONTHS_BALANCE_MAX_MAX,230982,75.1


In [85]:
features_to_be_dropped = features_to_be_dropped.index.to_list()
features_to_be_dropped

['ACTIVE_AMT_ANNUITY_MEAN',
 'ACTIVE_AMT_ANNUITY_MAX',
 'CLOSED_AMT_ANNUITY_MEAN',
 'CLOSED_AMT_ANNUITY_MAX',
 'ACTIVE_MONTHS_BALANCE_MIN_MIN',
 'ACTIVE_MONTHS_BALANCE_SIZE_MEAN',
 'ACTIVE_MONTHS_BALANCE_MAX_MAX']

In [101]:
df_cleaned = df.drop(features_to_be_dropped, axis=1)

In [102]:
df.shape

(307501, 210)

In [103]:
df_cleaned.shape

(307501, 203)

In [104]:
df_cleaned = df_cleaned.reset_index()

In [105]:
df_cleaned.replace([np.inf, -np.inf], np.nan, inplace=True)

In [109]:
columns = df_cleaned.columns.to_list()

In [110]:
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
df_cleaned = imputer.fit_transform(df_cleaned)

In [111]:
df_cleaned = pd.DataFrame(df_cleaned, columns=columns)
df_cleaned.head()

,index,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,CLOSED_AMT_CREDIT_SUM_DEBT_MEAN,CLOSED_AMT_CREDIT_SUM_DEBT_SUM,CLOSED_AMT_CREDIT_SUM_OVERDUE_MEAN,CLOSED_AMT_CREDIT_SUM_LIMIT_MEAN,CLOSED_AMT_CREDIT_SUM_LIMIT_SUM,CLOSED_CNT_CREDIT_PROLONG_SUM,CLOSED_MONTHS_BALANCE_MIN_MIN,CLOSED_MONTHS_BALANCE_MAX_MAX,CLOSED_MONTHS_BALANCE_SIZE_MEAN,CLOSED_MONTHS_BALANCE_SIZE_SUM
0,0.0,100002.0,1.0,0.0,0.0,0.0,202500.0,406597.5,24700.5,351000.0,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,-47.0000,0.00000,15.000000,90.000000
1,1.0,100003.0,0.0,0.0,1.0,0.0,270000.0,1293502.5,35698.5,1129500.0,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,-59.9837,-3.60576,33.457592,0.000000
2,2.0,100004.0,0.0,1.0,0.0,1.0,67500.0,135000.0,6750.0,135000.0,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,-59.9837,-3.60576,33.457592,0.000000
3,3.0,100006.0,0.0,0.0,1.0,0.0,135000.0,312682.5,29686.5,297000.0,...,2739.627971,9328.42334,3.859963,1089.433168,3677.142174,0.017027,-59.9837,-3.60576,33.457592,48.283109
4,4.0,100007.0,0.0,0.0,0.0,0.0,121500.0,513000.0,21865.5,513000.0,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,-59.9837,-3.60576,33.457592,0.000000


In [113]:
df_cleaned.shape

(307501, 204)

## **Saving Final Training Data** 

In [114]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [115]:
os.makedirs('/content/gdrive/MyDrive/Home_Credit_Default_Risk', exist_ok=True)  
df_cleaned.to_csv('/content/gdrive/MyDrive/Home_Credit_Default_Risk/df_cleaned.csv', index= False) 